In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from tensorflow.keras import backend
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import seaborn as sns

In [ ]:
from tensorflow import keras

In [ ]:
call_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/call_black_scholes.csv')
put_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/complete_data/put_black_scholes.csv')
call_df=call_df.set_index(['Date'])
call_df=call_df[call_df['Close']>1]

In [ ]:
call_df.head()

,Strike Price,Close,Expiration,Adj Close,return,volatility,Price,bs_call
Date,,,,,,,,
2013-12-31,6200,199.65,30,6304.0,0.002051,0.135503,8.82,188.621115
2013-12-31,6300,131.00,30,6304.0,0.002051,0.135503,8.82,124.059028
2013-12-31,6400,79.00,30,6304.0,0.002051,0.135503,8.82,75.195204
2013-12-31,6500,41.35,30,6304.0,0.002051,0.135503,8.82,41.683668
2013-12-31,6600,18.60,30,6304.0,0.002051,0.135503,8.82,21.015752


In [ ]:
spot_df=pd.read_csv('/content/drive/MyDrive/black_scholes_option_pricing/spot_price/^NSEI.csv')
spot_df.drop(['Open','High','Low','Close','Volume'],axis=1,inplace=True)
spot_df=spot_df.set_index(['Date'])
spot_df.dropna(inplace=True)

In [ ]:
spot_df.head()

,Adj Close
Date,
2013-12-02,6217.850098
2013-12-03,6201.850098
2013-12-04,6160.950195
2013-12-05,6241.100098
2013-12-06,6259.899902


In [ ]:
padded = np.insert(spot_df['Adj Close'].values, 0, np.array([np.nan] * 20))

In [ ]:
rolled = np.column_stack([np.roll(padded, i) for i in range(20)])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
rolled = rolled[~np.isnan(rolled).any(axis=1)]

In [ ]:
rolled = np.column_stack((spot_df[19:], rolled))

In [ ]:
price_history = pd.DataFrame(data=rolled)
price_history['date']=spot_df.index[19:]
price_history=price_history.set_index(['date'])

In [ ]:
joined = call_df.join(price_history,how='inner')

In [ ]:
joined.head()

,Strike Price,Close,Expiration,Adj Close,return,volatility,Price,bs_call,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
2013-12-31,6200,199.65,30,6304.0,0.002051,0.135503,8.82,188.621115,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098
2013-12-31,6300,131.00,30,6304.0,0.002051,0.135503,8.82,124.059028,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098
2013-12-31,6400,79.00,30,6304.0,0.002051,0.135503,8.82,75.195204,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098
2013-12-31,6500,41.35,30,6304.0,0.002051,0.135503,8.82,41.683668,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098
2013-12-31,6600,18.60,30,6304.0,0.002051,0.135503,8.82,21.015752,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098


In [ ]:
model_df.head()

,Strike Price,Close,Expiration,Adj Close,Price,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
2013-12-31,6200,199.65,30,6304.0,8.82,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098
2013-12-31,6300,131.00,30,6304.0,8.82,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098
2013-12-31,6400,79.00,30,6304.0,8.82,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098
2013-12-31,6500,41.35,30,6304.0,8.82,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098
2013-12-31,6600,18.60,30,6304.0,8.82,6304.0,6304.0,6291.100098,6313.799805,6278.899902,6268.399902,6284.5,6274.25,6166.649902,6217.149902,6139.049805,6154.700195,6168.399902,6237.049805,6307.899902,6332.850098,6363.899902,6259.899902,6241.100098,6160.950195,6201.850098


In [ ]:
model_df=joined.drop(['return','volatility','bs_call'],axis=1)

In [ ]:
def split_data(df):
  call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(df.drop(['Close'],axis=1).values,df['Close'].values,test_size=0.01,random_state=42)
  return call_X_train, call_X_test, call_y_train, call_y_test

In [ ]:
call_X_train, call_X_test, call_y_train, call_y_test=split_data(model_df)

In [ ]:
call_X_train.shape

(53925, 25)

In [ ]:
N_TIMESTEPS = 20

In [ ]:
call_X_train = [call_X_train[:, -N_TIMESTEPS:].reshape(call_X_train.shape[0], N_TIMESTEPS, 1), call_X_train[:, :4]]
call_X_test = [call_X_test[:, -N_TIMESTEPS:].reshape(call_X_test.shape[0], N_TIMESTEPS, 1), call_X_test[:, :4]]

In [ ]:
call_X_train

2

In [ ]:
layers = 4
n_timesteps = 20
features = 4
n_batch = 409
n_epochs = 100

In [ ]:
def make_model():
    close_history = Input((n_timesteps, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(LSTM(units=8, input_shape=(n_timesteps, 1), return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=False))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(1, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)


In [ ]:
model = make_model()

In [ ]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 8)            1952        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 12)           0           sequential[0][0]                 
                                                                 input_2[0][0]                

In [ ]:
model.compile(optimizer=Adam(lr=1e-4), loss='mse')
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

In [ ]:
y_pred=model.predict(call_X_test)

In [ ]:
y_pred.shape

(545, 1)

In [ ]:
x=np.array(call_y_test).reshape(-1,1)
#y=np.array(y_pred).reshape(-1,1)
reg=LinearRegression().fit(x,y_pred)
mae= mean_absolute_error(x,y_pred)
print(reg.score(x,y_pred),mae)

0.9731665577629489 171.63955107207696


In [ ]:
layers = 4
n_timesteps = 20
features = 4
n_batch = 409
n_epochs = 100

In [ ]:
import kerastuner as kt

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 6.0MB/s 


In [ ]:
def model(hp):
    close_history = Input((n_timesteps, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(LSTM(units=8, input_shape=(n_timesteps, 1), return_sequences=True))
    
    for i in range(hp.Int('num_layers',2,6)):
      lstm.add(LSTM(units=hp.Int('units_'+str(i),min_value=4,max_value=16,step=2),return_sequences=True))

    
    lstm.add(LSTM(units=8,return_sequences=False))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(1, activation='relu')(connect)

    new_model= Model(inputs=[close_history, input2], outputs=predict)
    new_model.compile(optimizer=Adam(hp.Choice('learning_rate', [1e-3, 1e-4,1e-5,1e-6])), loss='mse',metrics=['mse'])
    return new_model



In [ ]:
tuner = kt.Hyperband(
    model,
    objective='mse',
    max_epochs=10,seed=40,
    factor=3)

In [ ]:
tuner.search_space_summary()


Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 16, 'step': 2, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 16, 'step': 2, 'sampling': None}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05, 1e-06], 'ordered': True}


In [ ]:
tuner.search(call_X_train, call_y_train,epochs=10,validation_split=0.1)

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]


In [ ]:
best_model=tuner.hypermodel.build(best_hps)
best_model.fit(call_X_train, call_y_train, epochs=20, validation_split=0.1)

Epoch 1/20
1517/1517 [==============================] - 34s 17ms/step - loss: 153768.4844 - mse: 153768.4844 - val_loss: 29446.0762 - val_mse: 29446.0762
Epoch 2/20
1517/1517 [==============================] - 23s 15ms/step - loss: 28109.8066 - mse: 28109.8066 - val_loss: 20817.9453 - val_mse: 20817.9453
Epoch 3/20
1517/1517 [==============================] - 23s 15ms/step - loss: 23508.1562 - mse: 23508.1562 - val_loss: 20373.5273 - val_mse: 20373.5273
Epoch 4/20
1517/1517 [==============================] - 23s 15ms/step - loss: 21193.4375 - mse: 21193.4375 - val_loss: 23265.5762 - val_mse: 23265.5762
Epoch 5/20
1517/1517 [==============================] - 23s 15ms/step - loss: 18571.7812 - mse: 18571.7812 - val_loss: 10176.6963 - val_mse: 10176.6963
Epoch 6/20
1517/1517 [==============================] - 23s 15ms/step - loss: 16933.9863 - mse: 16933.9863 - val_loss: 12996.6738 - val_mse: 12996.6738
Epoch 7/20
1517/1517 [==============================] - 23s 15ms/step - loss: 15850.50

In [ ]:
best_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 8)            6224        input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 12)           0           sequential[0][0]                 
                                                                 input_2[0][0]                

In [ ]:
y_pred_new=best_model.predict(call_X_test)
y_pred_new.shape

(545, 1)

In [ ]:
y_pred_new

In [ ]:
y_pred_train=best_model.predict(call_X_train)

In [ ]:
y_pred_train.shape

(53925, 1)

In [ ]:
call_X_train_tm, call_X_test_tm, call_y_train_tm, call_y_test_tm = train_test_split(call_df.drop(['Close'],axis=1),call_df['Close'],test_size=0.01,random_state=42)

In [ ]:
call_X_test_tm

,Strike Price,Expiration,Adj Close,return,volatility,Price,bs_call
Date,,,,,,,
2020-02-10,13000,17,12031.500000,-0.005526,0.146067,6.440,1.204041
2015-02-20,8700,34,8833.599609,-0.006936,0.122263,7.695,250.668603
2019-06-18,11300,9,11691.500000,0.001658,0.129754,6.808,414.200121
2017-01-11,8100,14,8380.650391,0.011106,0.110837,6.391,303.894066
2020-02-12,12900,15,12201.200195,0.007706,0.151088,6.473,6.496768
...,...,...,...,...,...,...,...
2020-03-09,11900,17,10451.450195,-0.048956,0.245978,6.066,1.647730
2017-04-07,9100,20,9198.299805,-0.006872,0.095214,6.819,163.875642
2016-03-23,6700,8,7716.500000,0.000207,0.174305,7.507,1027.514914


In [ ]:
r2_score(list (call_X_test_tm["bs_call"]),y_pred_new)

0.934070188342824

In [ ]:
x=np.array(call_y_test).reshape(-1,1)
y=np.array(y_pred).reshape(-1,1)
reg=LinearRegression().fit(x,y_pred_new)
mae= mean_absolute_error(x,y_pred_new)
print(reg.score(x,y_pred_new),mae)

0.9631888609945654 91.24745231387811


In [ ]:
r2_score(call_y_train,y_pred_train)


0.9102510619288696

In [ ]:
def call_pricing(df):
  S=df['Adj Close']
  K= df["Strike Price"]
  t=df.Expiration/365
  r=df["Price"]/100
  sigma=df["volatility"]
  d1=(np.log(S/K)+(r+(sigma**2)/2)*t)/(sigma*(t**0.5))
  d2=d1-sigma*(t**0.5)
  Blackscholes_pred=  S*norm.cdf(d1)-K*np.exp(-r * t)*norm.cdf(d2)
  return Blackscholes_pred